# Silent Hunger Discovery Engine
## Notebook 02: Data Acquisition & Pre-processing

---

**Scope:** Pakistan & India (2010–2025)

This notebook defines the *structure* for data loading and cleaning. No analysis is performed here.

# Phase 1:
### 1.1: Environment Setup & Library Imports

In [1]:
# Core libraries
import pandas as pd
import zipfile
import numpy as np
import os 


# File handling
from pathlib import Path

# Settings
pd.set_option('display.max_columns', None)


## 1.2: Raw Data Ingestion


In [2]:
# --- STEP 1: Exact Filenames (As per your folder list) ---
FILE_INFLATION = 'worldbank_food_cpi_food.csv'
FILE_ANEMIA = 'who_anemia_women_15_49.csv'
FILE_STUNTING = 'unicef_stunting_latest.xlsx'
FILE_WASTING = 'unicef_wasting_latest.xlsx'

# --- STEP 2: Loading Logic with Error Handling ---
try:
    # 1. World Bank Inflation (Food CPI)
    try:
        # Checking if it's a ZIP disguised as CSV
        with zipfile.ZipFile(FILE_INFLATION, 'r') as z:
            data_file = [f for f in z.namelist() if 'Data.csv' in f][0]
            df_inflation_raw = pd.read_csv(z.open(data_file))
            print(f"✅ Loaded {FILE_INFLATION} (from Zip archive)")
    except:
        # If it's a normal CSV
        df_inflation_raw = pd.read_csv(FILE_INFLATION)
        print(f"✅ Loaded {FILE_INFLATION} (Standard CSV)")

    # 2. WHO Anemia
    df_anemia_raw = pd.read_csv(FILE_ANEMIA)
    print(f"✅ Loaded {FILE_ANEMIA}")

    # 3. UNICEF Stunting (Excel)
    # Hum 'Stunting Prevalence' sheet load kar rahe hain
    df_stunting_raw = pd.read_excel(FILE_STUNTING, sheet_name='Stunting Prevalence')
    print(f"✅ Loaded {FILE_STUNTING}")

    # 4. UNICEF Wasting (Excel)
    # Hum 'Primary Data' sheet load kar rahe hain
    df_wasting_raw = pd.read_excel(FILE_WASTING, sheet_name='Primary Data')
    print(f"Loaded {FILE_WASTING}")

    print("\n SUCCESS! All datasets are now in memory.")

except Exception as e:
    print(f"\n STILL AN ERROR: {e}")
    print("Tip: Make sure the spelling of the file in the code matches your folder exactly!")

# --- QUICK PREVIEW ---
if 'df_stunting_raw' in locals():
    print("\n--- Stunting Data Head ---")
    display(df_stunting_raw.head(3))

✅ Loaded worldbank_food_cpi_food.csv (from Zip archive)
✅ Loaded who_anemia_women_15_49.csv
✅ Loaded unicef_stunting_latest.xlsx
Loaded unicef_wasting_latest.xlsx

 SUCCESS! All datasets are now in memory.

--- Stunting Data Head ---


,Indicator,Measure,ISO Code,Country or Area,Year,Both Sexes - Point Estimates,Both Sexes - Lower Limit,Both Sexes - Upper Limit,Male - Point Estimate,Male - Lower Limit,Male - Upper Limit,Female - Point Estimate,Female - Lower Limit,Female - Upper Limit,Note
0,Stunting,Prevalence,AFG,Afghanistan,2000,55.5,52.3,58.6,55.9,52.7,59.2,55.1,51.8,58.3,NaN
1,Stunting,Prevalence,AFG,Afghanistan,2001,55.2,52.1,58.3,55.6,52.6,58.6,54.7,51.6,57.9,NaN
2,Stunting,Prevalence,AFG,Afghanistan,2002,54.4,51.5,57.2,54.8,52.0,57.7,53.9,51.0,56.8,NaN


## 1.3: Concluding Step - Full Data Inspection

In [3]:
# --- STEP 3: Structural Inspection of all Datasets ---

print("--- INFLATION DATA (World Bank) ---")
print(f"Shape: {df_inflation_raw.shape}")
print("Columns Available:", df_inflation_raw.columns.tolist())

print("\n--- ANEMIA DATA (WHO) ---")
print(f"Shape: {df_anemia_raw.shape}")
# Yahan hum saare columns print karenge taake KeyError na aaye
print("Columns Available:", df_anemia_raw.columns.tolist())

print("\n--- STUNTING DATA (UNICEF 2025) ---")
print(f"Shape: {df_stunting_raw.shape}")
# Humne preview mein dekha tha ke ye columns sahi hain
cols_stunting = ['ISO Code', 'Year', 'Both Sexes - Point Estimates']
print(df_stunting_raw[cols_stunting].head(2))

print("\n--- WASTING DATA (UNICEF 2025) ---")
print(f"Shape: {df_wasting_raw.shape}")
# Wasting mein 'Year*' aur 'Wasting' check karte hain
cols_wasting = ['ISO code', 'Year*', 'Wasting']
print(df_wasting_raw[cols_wasting].head(2))

print("\n✅ Phase 1 Inspection Done. Now we know the exact column names!")

--- INFLATION DATA (World Bank) ---
Shape: (7, 14)
Columns Available: ['Country Name', 'Country Code', 'Series Name', 'Series Code', '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]', '2020 [YR2020]', '2021 [YR2021]', '2022 [YR2022]', '2023 [YR2023]', '2024 [YR2024]']

--- ANEMIA DATA (WHO) ---
Shape: (17555, 16)
Columns Available: ['IND_ID', 'IND_CODE', 'IND_UUID', 'IND_PER_CODE', 'DIM_TIME', 'DIM_TIME_TYPE', 'DIM_GEO_CODE_M49', 'DIM_GEO_CODE_TYPE', 'DIM_PUBLISH_STATE_CODE', 'IND_NAME', 'GEO_NAME_SHORT', 'DIM_SEX', 'DIM_PREGNANCYSTATUS', 'RATE_PER_100_N', 'RATE_PER_100_NL', 'RATE_PER_100_NU']

--- STUNTING DATA (UNICEF 2025) ---
Shape: (5050, 15)
  ISO Code  Year Both Sexes - Point Estimates
0      AFG  2000                         55.5
1      AFG  2001                         55.2

--- WASTING DATA (UNICEF 2025) ---
Shape: (3222, 37)
  ISO code  Year*  Wasting
0      AFG   1997     18.2
1      AFG   2004      8.6

✅ Phase 1 Inspection Done. Now we kn

## 1.4: Deep Dive - Top 5 Rows Inspection

In [4]:
# --- STEP 4: Deep Inspection of 5 rows ---

print("--- INFLATION (Wide Format Check) ---")
# Is mein hum dekhenge ke years kaise likhe hain
display(df_inflation_raw.head(5))

print("\n--- ANEMIA (Column Value Check) ---")
# Is mein hum GEO_NAME_SHORT check karenge ke Pakistan/India kaise likha hai
display(df_anemia_raw[['GEO_NAME_SHORT', 'DIM_TIME', 'RATE_PER_100_N']].head(5))

print("\n--- STUNTING (Point Estimates Check) ---")
display(df_stunting_raw[['ISO Code', 'Country or Area', 'Year', 'Both Sexes - Point Estimates']].head(5))

print("\n--- WASTING (Year* Check) ---")
# Wasting mein 'Year*' aur 'Wasting' ki values dekhte hain
display(df_wasting_raw[['ISO code', 'Country and areas', 'Year*', 'Wasting']].head(5))

--- INFLATION (Wide Format Check) ---


,Country Name,Country Code,Series Name,Series Code,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023],2024 [YR2024]
0,India,IND,Consumer price index (2010 = 100),FP.CPI.TOTL,146.790502,154.054013,159.181198,165.451069,171.621576,182.988823,192.378725,205.266241,216.862025,227.603278
1,Pakistan,PAK,Consumer price index (2010 = 100),FP.CPI.TOTL,145.282483,150.752541,156.911346,164.879394,182.320933,200.078979,219.078900,262.618334,343.421079,386.803857
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



--- ANEMIA (Column Value Check) ---


,GEO_NAME_SHORT,DIM_TIME,RATE_PER_100_N
0,Cabo Verde,2000,50.0
1,Central African Republic,2000,50.3
2,Asia,2000,31.5
3,Central Asia,2000,39.8
4,Sri Lanka,2000,31.2



--- STUNTING (Point Estimates Check) ---


,ISO Code,Country or Area,Year,Both Sexes - Point Estimates
0,AFG,Afghanistan,2000,55.5
1,AFG,Afghanistan,2001,55.2
2,AFG,Afghanistan,2002,54.4
3,AFG,Afghanistan,2003,53.4
4,AFG,Afghanistan,2004,52.4



--- WASTING (Year* Check) ---


,ISO code,Country and areas,Year*,Wasting
0,AFG,AFGHANISTAN,1997,18.2
1,AFG,AFGHANISTAN,2004,8.6
2,AFG,AFGHANISTAN,2004,7.9
3,AFG,AFGHANISTAN,2004,9.3
4,AFG,AFGHANISTAN,2013,9.5


In this phase, all four external datasets were successfully ingested and inspected in their raw form to understand structure, variable naming, time formats, and metadata placement. A targeted row-level inspection confirmed differences in data layouts (wide vs. long, survey-based vs. annual), guiding the harmonization strategy for subsequent phases without applying any transformations at this stage.

# PHASE 2: RAW DATA INVENTORY & STORAGE

In [5]:
# 1. Create the directory structure
os.makedirs('data/raw', exist_ok=True)

# 2. Save the loaded dataframes to the raw folder for future use
df_anemia_raw.to_csv('data/raw/who_anemia_women_15_49.csv', index=False)
df_inflation_raw.to_csv('data/raw/worldbank_food_cpi_food.csv', index=False)
df_stunting_raw.to_csv('data/raw/unicef_stunting_children_u5.csv', index=False)
df_wasting_raw.to_csv('data/raw/unicef_wasting_children_u5.csv', index=False)

# 3. Final Inventory Verification
print("PHASE 2: FINAL INVENTORY CHECK")
print("-" * 30)
files_in_raw = os.listdir('data/raw')
expected_files = [
    'worldbank_food_cpi_food.csv',
    'who_anemia_women_15_49.csv',
    'unicef_stunting_children_u5.csv',
    'unicef_wasting_children_u5.csv'
]

for file in expected_files:
    if file in files_in_raw:
        print(f" Verified: {file}")
    else:
        print(f" Missing: {file}")

PHASE 2: FINAL INVENTORY CHECK
------------------------------
 Verified: worldbank_food_cpi_food.csv
 Verified: who_anemia_women_15_49.csv
 Verified: unicef_stunting_children_u5.csv
 Verified: unicef_wasting_children_u5.csv


In this phase, all identified external datasets were successfully loaded and securely stored in their original raw form without any modifications. This ensures data integrity and provides a stable foundation for all subsequent cleaning, harmonization, and analysis steps.

#  Phase 2: Data Cleaning & Harmonization

##  2.1: World Bank Inflation (Food CPI) Fix

In [6]:
# --- PHASE 2.1: CLEANING WORLD BANK INFLATION ---

# 1. Sirf Pakistan aur India filter karna aur faltu empty rows hatana
df_inf_clean = df_inflation_raw[df_inflation_raw['Country Code'].isin(['PAK', 'IND'])].dropna(subset=['Country Code'])

# 2. Reshaping (Melting) - Wide format ko vertical karna
year_cols = [c for c in df_inf_clean.columns if '[' in c] # 2015 [YR2015] types columns

df_inf_long = pd.melt(df_inf_clean, 
                      id_vars=['Country Name', 'Country Code'], 
                      value_vars=year_cols, 
                      var_name='Year_Raw', value_name='Value')

# 3. Clean Year: '2015 [YR2015]' se sirf '2015' nikalna
df_inf_long['Year'] = df_inf_long['Year_Raw'].str.split(' ').str[0].astype(int)

# 4. Standardizing for Master Schema
df_inflation_final = df_inf_long[['Country Name', 'Year', 'Value']].copy()
df_inflation_final['Indicator'] = 'Food_CPI'
df_inflation_final.columns = ['Country', 'Year', 'Value', 'Indicator']

print("Step 2.1 Complete: Inflation Data Cleaned.")
display(df_inflation_final.head(3))

Step 2.1 Complete: Inflation Data Cleaned.


,Country,Year,Value,Indicator
0,India,2015,146.790502,Food_CPI
1,Pakistan,2015,145.282483,Food_CPI
2,India,2016,154.054013,Food_CPI


In this phase, the World Bank Food CPI data was reshaped from a wide, year-based format into a standardized long format to enable time-series analysis. Column names and year values were cleaned and unified to align with the common schema planned for cross-dataset integration in later phases.

## 2.2: WHO Anemia Data Cleaning

In [7]:
# --- PHASE 2.2: CLEANING WHO ANEMIA DATA ---

# 1. Filter only Pakistan and India
# Note: WHO data mein 'India' aur 'Pakistan' likha hota hai
df_anemia_clean = df_anemia_raw[df_anemia_raw['GEO_NAME_SHORT'].isin(['India', 'Pakistan'])].copy()

# 2. Select only necessary columns
# DIM_TIME is Year, RATE_PER_100_N is the prevalence value
df_anemia_final = df_anemia_clean[['GEO_NAME_SHORT', 'DIM_TIME', 'RATE_PER_100_N']].copy()

# 3. Add Indicator label and Rename to match Master Schema
df_anemia_final['Indicator'] = 'Anemia_Prevalence'
df_anemia_final.columns = ['Country', 'Year', 'Value', 'Indicator']

# 4. Sorting for better view
df_anemia_final = df_anemia_final.sort_values(by=['Country', 'Year'])

print("Step 2.2 Complete: Anemia Data Cleaned.")
display(df_anemia_final.head(3))

Step 2.2 Complete: Anemia Data Cleaned.


,Country,Year,Value,Indicator
136,India,2000,50.1,Anemia_Prevalence
5991,India,2000,51.6,Anemia_Prevalence
11848,India,2000,50.2,Anemia_Prevalence


In this step, the WHO anemia dataset was filtered to focus on the study’s target countries and reduced to only analytically relevant variables. The data was then standardized to match the master schema, ensuring consistency with other harmonized indicators.

##  2.3: UNICEF Stunting & Wasting

In [8]:
# --- PHASE 2.3: CLEANING UNICEF NUTRITION DATA ---

# --- 1. CLEANING STUNTING ---
# Filter for Pakistan and India
df_stunting_clean = df_stunting_raw[df_stunting_raw['ISO Code'].isin(['PAK', 'IND'])].copy()

# Select columns and Rename to Schema
# 'Both Sexes - Point Estimates' is the value column
df_stunting_final = df_stunting_clean[['Country or Area', 'Year', 'Both Sexes - Point Estimates']].copy()
df_stunting_final['Indicator'] = 'Child_Stunting'
df_stunting_final.columns = ['Country', 'Year', 'Value', 'Indicator']


# --- 2. CLEANING WASTING ---
# Filter for Pakistan and India (Note: small 'c' in 'ISO code' here)
df_wasting_clean = df_wasting_raw[df_wasting_raw['ISO code'].isin(['PAK', 'IND'])].copy()

# Select columns and Rename to Schema
# 'Year*' is the year, 'Wasting' is the value
df_wasting_final = df_wasting_clean[['Country and areas', 'Year*', 'Wasting']].copy()
df_wasting_final['Indicator'] = 'Child_Wasting'
df_wasting_final.columns = ['Country', 'Year', 'Value', 'Indicator']

print("Step 2.3 Complete: UNICEF Stunting & Wasting Cleaned.")
print("\n--- Stunting Sample ---")
display(df_stunting_final.head(2))
print("\n--- Wasting Sample ---")
display(df_wasting_final.head(2))

Step 2.3 Complete: UNICEF Stunting & Wasting Cleaned.

--- Stunting Sample ---


,Country,Year,Value,Indicator
2025,India,2000,50.0,Child_Stunting
2026,India,2001,49.6,Child_Stunting



--- Wasting Sample ---


,Country,Year,Value,Indicator
1105,INDIA,1989,20.3,Child_Wasting
1106,INDIA,1991,20.0,Child_Wasting


In this step, UNICEF stunting and wasting datasets were filtered for Pakistan and India and reduced to analytically relevant variables. Key indicators were standardized into a common schema (Country, Year, Value, Indicator) to ensure consistency with other health and economic datasets.

# Phase 3: Master Harmonization & Schema Alignment

### 3.1 The Master Code

In [9]:
# --- PHASE 3.1: THE MASTER CONCATENATION (FIXED) ---

# 1. Sab dataframes ko ek list mein jama karna
frames = [df_inflation_final, df_anemia_final, df_stunting_final, df_wasting_final]

# 2. Combine all tables vertically
master_df = pd.concat(frames, ignore_index=True)

# 3. Clean Country Names (Normalization)
master_df['Country'] = master_df['Country'].str.strip().str.title()

# --- THE FIX STARTS HERE ---
# 4. Force 'Value' to be Numeric
# 'coerce' ka matlab hai agar koi ghalat text ho toh usay NaN (empty) kar do, error na do
master_df['Value'] = pd.to_numeric(master_df['Value'], errors='coerce')

# 5. Drop any rows where Value became NaN after conversion
master_df = master_df.dropna(subset=['Value'])
# --- THE FIX ENDS HERE ---

# 6. Handle Duplicates & Multiple Surveys (Ab error nahi ayega!)
master_df = master_df.groupby(['Country', 'Year', 'Indicator'])['Value'].mean().reset_index()

# 7. Lock Timeline (2010 - 2025)
master_df = master_df[(master_df['Year'] >= 2010) & (master_df['Year'] <= 2025)]

# 8. Sorting
master_df = master_df.sort_values(by=['Country', 'Indicator', 'Year']).reset_index(drop=True)

print("--- MASTER ENGINE READY ---")
print(f"Total Rows: {len(master_df)}")
display(master_df.head(10))

--- MASTER ENGINE READY ---
Total Rows: 85


,Country,Year,Indicator,Value
0,India,2010,Anemia_Prevalence,50.033333
1,India,2011,Anemia_Prevalence,50.000000
2,India,2012,Anemia_Prevalence,49.933333
3,India,2013,Anemia_Prevalence,49.966667
4,India,2014,Anemia_Prevalence,50.033333
5,India,2015,Anemia_Prevalence,50.066667
6,India,2016,Anemia_Prevalence,50.133333
7,India,2017,Anemia_Prevalence,50.266667
8,India,2018,Anemia_Prevalence,50.366667
9,India,2019,Anemia_Prevalence,50.466667


### 3.2: Missing Data Interpolation

In [10]:
# --- PHASE 3.2: MISSING YEARS & INDICATORS CHECK ---

# 1. Dekhte hain har mulk ke paas kaunse indicators hain
print("Indicators available per country:")
display(master_df.groupby('Country')['Indicator'].unique())

# 2. Check for Missing Years (2010 - 2024 total 15 years hone chahiye)
year_counts = master_df.groupby(['Country', 'Indicator'])['Year'].count().reset_index()
year_counts.columns = ['Country', 'Indicator', 'Years_Count']

print("\nData points per Indicator (Goal is ~15 years):")
display(year_counts)

Indicators available per country:


Country
India       [Anemia_Prevalence, Child_Stunting, Child_Wast...
Pakistan    [Anemia_Prevalence, Child_Stunting, Child_Wast...
Name: Indicator, dtype: object


Data points per Indicator (Goal is ~15 years):


,Country,Indicator,Years_Count
0,India,Anemia_Prevalence,14
1,India,Child_Stunting,15
2,India,Child_Wasting,4
3,India,Food_CPI,10
4,Pakistan,Anemia_Prevalence,14
5,Pakistan,Child_Stunting,15
6,Pakistan,Child_Wasting,3
7,Pakistan,Food_CPI,10


### 3.3: Interpolation (Filling the Gaps)

In [11]:
# --- PHASE 3.3: FILLING THE TEMPORAL GAPS ---

# 1. Pehle ek 'Full Timeline' banate hain (2010-2025)
all_years = range(2010, 2026)
countries = ['India', 'Pakistan']
indicators = ['Anemia_Prevalence', 'Child_Stunting', 'Child_Wasting', 'Food_CPI']

import itertools
full_grid = pd.DataFrame(list(itertools.product(countries, all_years, indicators)), 
                         columns=['Country', 'Year', 'Indicator'])

# 2. Apne data ko is grid ke saath merge karein
master_full = pd.merge(full_grid, master_df, on=['Country', 'Year', 'Indicator'], how='left')

# 3. Interpolation: Gaps ko fill karna
# Linear interpolation as-paas ki values ko dekh kar beech ki value nikaalti hai
master_full['Value'] = master_full.groupby(['Country', 'Indicator'])['Value'].transform(
    lambda x: x.interpolate(method='linear', limit_direction='both')
)

print("--- INTERPOLATION COMPLETE ---")
print(f"Total rows now: {len(master_full)} (16 years x 2 countries x 4 indicators)")
display(master_full.head(10))

--- INTERPOLATION COMPLETE ---
Total rows now: 128 (16 years x 2 countries x 4 indicators)


,Country,Year,Indicator,Value
0,India,2010,Anemia_Prevalence,50.033333
1,India,2010,Child_Stunting,43.800000
2,India,2010,Child_Wasting,15.066667
3,India,2010,Food_CPI,146.790502
4,India,2011,Anemia_Prevalence,50.000000
5,India,2011,Child_Stunting,42.700000
6,India,2011,Child_Wasting,15.066667
7,India,2011,Food_CPI,146.790502
8,India,2012,Anemia_Prevalence,49.933333
9,India,2012,Child_Stunting,41.700000


A complete country–year–indicator grid (2010–2025) was constructed to ensure structural consistency across all variables. Missing values were then filled using linear interpolation within each country and indicator group, producing a balanced master dataset of 128 observations ready for analytical modeling.

# Phase 4: Data Quality Checks



### 4.1: Missing Values & Temporal Gaps (Final Verification)

In [12]:
# --- PHASE 4.1: MISSING VALUE ANALYSIS ---
# Checking for any remaining null values after interpolation
missing_data_count = master_full.isnull().sum()

print("Checking for null values post-interpolation:")
print(missing_data_count)

# Validating row count: Expected 128 rows (2 countries * 16 years * 4 indicators)
expected_rows = 128
actual_rows = len(master_full)

if actual_rows == expected_rows:
    print(f"Temporal integrity verified: {actual_rows} rows generated.")
else:
    print(f"Warning: Row count mismatch. Expected {expected_rows}, found {actual_rows}.")

Checking for null values post-interpolation:
Country      0
Year         0
Indicator    0
Value        0
dtype: int64
Temporal integrity verified: 128 rows generated.


### 4.2: Indicator Definition & Range Consistency

In [13]:
# --- PHASE 4.2: RANGE AND DEFINITION CONSISTENCY ---
# Statistical profiling to detect logical inconsistencies in data definitions
range_stats = master_full.groupby('Indicator')['Value'].agg(['min', 'max', 'mean', 'std'])

print("Indicator Range Statistics:")
print(range_stats)

# Logical Validation: Ensure no negative values exist in health/economic indicators
negative_values = master_full[master_full['Value'] < 0]

if negative_values.empty:
    print("Logical Check: All indicator values are non-negative.")
else:
    print("Error: Negative values detected in the following records:")
    print(negative_values)

Indicator Range Statistics:
                          min         max        mean        std
Indicator                                                       
Anemia_Prevalence   44.233333   51.433333   47.750000   2.850329
Child_Stunting      32.800000   44.900000   37.865625   4.135730
Child_Wasting        7.100000   20.766667   13.263542   4.868371
Food_CPI           145.282483  386.803857  190.909917  67.379752
Logical Check: All indicator values are non-negative.


### 4.3: Outlier Detection (Statistical Variance)

In [14]:
# --- PHASE 4.3: OUTLIER DETECTION VIA VARIANCE ANALYSIS ---
# Calculating Year-over-Year (YoY) percentage change to identify anomalies
master_full['YoY_Change'] = master_full.groupby(['Country', 'Indicator'])['Value'].pct_change()

# Defining threshold for potential outliers (e.g., >30% change in one year)
outlier_threshold = 0.3
detected_outliers = master_full[master_full['YoY_Change'].abs() > outlier_threshold]

print(f"Outlier Analysis (Threshold: {outlier_threshold*100}% YoY Change):")
if detected_outliers.empty:
    print("No significant outliers detected based on the variance threshold.")
else:
    print("Potential outliers/shocks detected:")
    print(detected_outliers[['Country', 'Year', 'Indicator', 'Value', 'YoY_Change']])

Outlier Analysis (Threshold: 30.0% YoY Change):
Potential outliers/shocks detected:
      Country  Year      Indicator       Value  YoY_Change
22      India  2015  Child_Wasting   20.766667    0.378319
119  Pakistan  2023       Food_CPI  343.421079    0.307681


Year-over-year percentage change was calculated for each country and indicator to detect abnormal fluctuations using a 30% variance threshold. The analysis identified notable shocks in India’s child wasting (2015) and Pakistan’s Food CPI (2023), suggesting potential structural or economic disruptions requiring contextual validation.


# Phase 5: Digital Signal Data (Deferred)

Google Trends integration is postponed due to API reliability concerns.
Behavioral signals will be added later using cached or offline data.

# Phase 6: Output Specification



In [15]:
# --- PHASE 6: OUTPUT SPECIFICATION & EXPORT ---
# Exporting the finalized, interpolated, and validated dataset for SHDE Engine
output_file = 'shde_master_engine_v1.csv'

# Dropping auxiliary columns like 'YoY_Change' to keep the master file clean
final_export_df = master_full.drop(columns=['YoY_Change'])

# Saving to CSV
final_export_df.to_csv(output_file, index=False)

print(f"Dataset Exported Successfully: {output_file}")
print(f"Final Shape: {final_export_df.shape}")
print("Notebook 02: Data Acquisition & Pre-processing is officially CLOSED.")# --- PHASE 5: OUTPUT SPECIFICATION & EXPORT ---
# Exporting the finalized, interpolated, and validated dataset for SHDE Engine
output_file = 'shde_master_engine_v1.csv'

# Dropping auxiliary columns like 'YoY_Change' to keep the master file clean
final_export_df = master_full.drop(columns=['YoY_Change'])

# Saving to CSV
final_export_df.to_csv(output_file, index=False)

print(f"Dataset Exported Successfully: {output_file}")
print(f"Final Shape: {final_export_df.shape}")
print("Notebook 02: Data Acquisition & Pre-processing is officially CLOSED.")

Dataset Exported Successfully: shde_master_engine_v1.csv
Final Shape: (128, 4)
Notebook 02: Data Acquisition & Pre-processing is officially CLOSED.
Dataset Exported Successfully: shde_master_engine_v1.csv
Final Shape: (128, 4)
Notebook 02: Data Acquisition & Pre-processing is officially CLOSED.


## Final Summary

Notebook 02 established a structured and reproducible data pipeline for Pakistan and India (2010–2025) by integrating datasets from the World Bank, WHO, and UNICEF. All raw files were securely stored and verified before undergoing systematic inspection and schema alignment. Each dataset was cleaned and reduced to a standardized format consisting of Country, Year, Indicator, and Value to ensure cross-source compatibility. A complete country–year–indicator grid was constructed, and missing values were filled using linear interpolation to maintain temporal continuity. Data quality checks, including year-over-year variance analysis, were applied to detect potential statistical shocks or anomalies. The final output is a harmonized, analysis-ready master dataset that forms the technical foundation for subsequent modeling and signal development in the Silent Hunger Discovery Engine.

## Closing

This notebook establishes a reproducible data foundation.
No modeling or inference should begin before this phase is completed.